## Analysis of the species distribution in MSA

### 1. Get the species names from the MSA, based on the downloaded UniRef100 gzipped file, and store them in an sqlite database.

In [3]:
import gzip
import sqlite3
import re
from tqdm import tqdm  # This is just for the progress bar (optional: install it with `pip install tqdm`)

# Get MSA file from user input
msa_file = input("Enter the path to your MSA file (.a3m file): ").strip()

# Collect UniRef IDs from our MSA
uniref_ids_in_msa = set()

try:
    # First, count the total lines for the progress bar
    with open(msa_file, "r") as f:
        total_lines = sum(1 for _ in f)
    
    # Now process with progress bar
    with open(msa_file, "r") as f:
        for line in tqdm(f, total=total_lines, desc="Processing MSA file"):
            # Valid IDs start with the below prefix
            if line.startswith(">UniRef100_"):
                uniref_id = line.strip()[1:].split()[0]  # This will extract UniRef100_A0A1B0GTX2 for example
                uniref_ids_in_msa.add(uniref_id)

    print(f"Found {len(uniref_ids_in_msa)} UniRef IDs in MSA.")
    
except FileNotFoundError:
    print(f"Error: File '{msa_file}' not found. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred while processing the file: {e}")

Processing MSA file: 100%|██████████| 14175/14175 [00:00<00:00, 1907847.74it/s]

Found 6423 UniRef IDs in MSA.


### 2. Build the lookup table (SQLite)

In [4]:
def get_user_paths():
    """Get database and UniRef file paths from user input"""
    db_path = input("Enter the path for the SQLite database file: ").strip()
    uniref_gz_path = input("Enter the path for the UniRef100 gzipped FASTA file: ").strip()
    return db_path, uniref_gz_path

# Initialise the SQLite database. SQLite file will serve as the lookup table
def init_db(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS uniref_species (
            uniref_id TEXT PRIMARY KEY,
            species TEXT
        )
    """)
    conn.commit()
    return conn

# Get paths from user input
db_path, uniref_gz_path = get_user_paths()

# Initialise the database connection
conn = init_db(db_path)

# Scan UniRef100.fasta.gz ONLY for IDs in your MSA
found_ids = set()

def extract_species(header):
    tax_match = re.search(r'Tax=([^=]+?)\s+TaxID=', header)
    if tax_match:
        species = tax_match.group(1).strip()
        species = re.sub(r'[\d_]+$', '', species).strip()
        return species or "Unknown"
    return "Unknown"

# Stream gzip file line-by-line
with gzip.open(uniref_gz_path, "rt") as f:
    current_id = None
    for line in tqdm(f, desc="Scanning UniRef100.fasta.gz"):
        if line.startswith(">"):
            current_id = line.split()[0][1:]  # Extract UniRef100_A0A1B0GTX2
            if current_id in uniref_ids_in_msa:
                species = extract_species(line)
                conn.cursor().execute(
                    "INSERT OR IGNORE INTO uniref_species VALUES (?, ?)",
                    (current_id, species)
                )
                found_ids.add(current_id)
                # Early exit if all IDs are found
                if len(found_ids) == len(uniref_ids_in_msa):
                    break
    conn.commit()

print(f"Found species for {len(found_ids)}/{len(uniref_ids_in_msa)} UniRef IDs.")

Scanning UniRef100.fasta.gz: 3596596416it [22:08, 2706381.04it/s]

Found species for 5983/6423 UniRef IDs.
